<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/Single_Image_Super_Resolution_video_20220411.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Github git 클론해서 드라이브에 보관하기
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir /content/drive/MyDrive/Colab Notebooks/SISR
#cd "/content/drive/MyDrive/Colab Notebooks/SISR"
#!git clone https://github.com/cszn/KAIR.git

In [ ]:
#Github git 클론해서 가져와 사용하기
#!git clone https://github.com/cszn/KAIR.git

# BSRGAN을 활용한 SISR로 옛날 비디오 복원하기

In [ ]:
#Github git 클론해서 가져와 사용하기
!git clone https://github.com/cszn/BSRGAN.git

Cloning into 'BSRGAN'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (267/267), done.
remote: Total 535 (delta 161), reused 4 (delta 3), pack-reused 258
Receiving objects: 100% (535/535), 168.90 MiB | 31.37 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [ ]:
import torch
import cv2
import os
import sys
sys.path.append("/content/BSRGAN")
from utils import utils_image as util
from models.network_rrdbnet import RRDBNet as net
import main_download_pretrained_models

In [ ]:
!python /content/BSRGAN/main_download_pretrained_models.py --models "BSRGAN" --model_dir "/content/BSRGAN/model_zoo"
!python /content/BSRGAN/main_download_pretrained_models.py --models "BSRGAN.pth" --model_dir "/content/BSRGAN/model_zoo"

trying to download ['BSRGAN']
downloading [/content/BSRGAN/model_zoo/BSRGAN.pth] ...
done!
downloading [/content/BSRGAN/model_zoo/BSRNet.pth] ...
done!
downloading [/content/BSRGAN/model_zoo/BSRGANx2.pth] ...
done!
trying to download ['BSRGAN.pth']
already exists, skip downloading [BSRGAN.pth]


In [ ]:
#import urllib.request
# 다운받을 이미지 url
#url = "https://yosoro.com/wp/wp-content/uploads/2018/07/oldmitarai02.jpg"
# 이미지 요청 및 다운로드
#urllib.request.urlretrieve(url, "test.jpg")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving KakaoTalk_20220216_153909289.mp4 to KakaoTalk_20220216_153909289.mp4


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = net(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, sf=4)
model.load_state_dict(torch.load(os.path.join('/content/BSRGAN/model_zoo', 'BSRGAN.pth')), strict=True)
model.eval()

[3, 3, 64, 23, 32, 4]


RRDBNet(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), str

In [ ]:
for k, v in model.named_parameters():
  v.requires_grad = False
model = model.to(device)

In [ ]:
for VIDEO_PATH in uploaded.keys():
  cap = cv2.VideoCapture(VIDEO_PATH)
  
  fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
  out = cv2.VideoWriter('%s_output.mp4' % (VIDEO_PATH.split('.')[0]), fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 4), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 4)))
  n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  i = 0
  
  print(n_frames)

  cap.set(cv2.CAP_PROP_POS_FRAMES, i)

  while cap.isOpened():
    ret, img = cap.read()
    if not ret:
      break
    
    img_L = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_L = util.uint2tensor4(img_L)
    img_L = img_L.to(device)

    img_E = model(img_L)
    img_E = util.tensor2uint(img_E)
    img_E = cv2.cvtColor(img_E, cv2.COLOR_RGB2BGR)

    out.write(img_E)

    i += 1
    print('%d/%d' % (i, n_frames))

  out.release()
  cap.release()

324.0
1/324
2/324
3/324
4/324
5/324
6/324
7/324
8/324
9/324
10/324
11/324
12/324
13/324
14/324
15/324
16/324
17/324
18/324
19/324
20/324
21/324
22/324
23/324
24/324
25/324
26/324
27/324
28/324
29/324
30/324
31/324
32/324
33/324
34/324
35/324
36/324
37/324
38/324
39/324
40/324
41/324
42/324
43/324
44/324
45/324
46/324
47/324
48/324
49/324
50/324
51/324
52/324
53/324
54/324
55/324
56/324
57/324
58/324
59/324
60/324
61/324
62/324
63/324
64/324
65/324
66/324
67/324
68/324
69/324
70/324
71/324
72/324
73/324
74/324
75/324
76/324
77/324
78/324
79/324
80/324
81/324
82/324
83/324
84/324
